### torchsummaryの使い方

***

torchsummaryとは、特徴量マップのサイズを確認することができる。

In [18]:
import torch
import torch.nn as nn
from torchsummary import summary
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels= 16,
            kernel_size=3,
            stride=1
            )
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1
            )
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.conv3 = nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3,
            stride=1
            )
        self.gap = nn.AdaptiveAvgPool2d(1)
    def forward(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.gap(x)
        return x
model = SimpleCNN()
summary(model, input_size=(3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 222, 222]             448
       BatchNorm2d-2         [-1, 16, 222, 222]              32
              ReLU-3         [-1, 16, 222, 222]               0
         MaxPool2d-4         [-1, 16, 111, 111]               0
            Conv2d-5         [-1, 32, 109, 109]           4,640
       BatchNorm2d-6         [-1, 32, 109, 109]              64
              ReLU-7         [-1, 32, 109, 109]               0
         MaxPool2d-8           [-1, 32, 54, 54]               0
            Conv2d-9           [-1, 64, 52, 52]          18,496
AdaptiveAvgPool2d-10             [-1, 64, 1, 1]               0
Total params: 23,680
Trainable params: 23,680
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.57
Forward/backward pass size (MB): 30.29
Params size (MB): 0.09
Estimated To

Bad pipe message: %s [b'kjA\xf0\xbe[\x14\xc8\xb6 \x8e\xb8\x84\x14_\x8e\xe5v \xa3!W+1\xe7\x9aGF\x13yz\x04\xfc\x02\xb25J\x1dS\xbe\xf27\xf0\xd8Ue\xc1\xc8F\ne\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00', b'\x0c\x00\x00\t127.0.0.1']
Bad pipe message: %s [b'\x17\x01\xb9\xee\r\xa3\x84|\xd6\x97-\x05\\^C|O\xf2\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00:\x00\x89\xc0\x0f\xc0\x05\x005\x00\x84\xc0\x13\xc0\t\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x00C\x00B\xc0\x18\x004\x00\x9b\x00F\xc0\x0e\xc0\x04\x00/\x00\x96\x00A\x00\x07\xc0\x11\xc0\x07\xc0\x16\x00\x18\xc0\x0c\xc0\x02\x00\x05\x00\x04\xc0\x12\xc0\x08\x00\x16\x00\x13\x00\x10\x00\r\xc0\x17\x00\x1b\xc0\r\xc0\x03\x00\n\x00\x15\x00\x12\x00\x0f\x00\x0c\x00\x1a\x00\t\x00\x14\x00\x11\x00\x19\x00', b'\x06\x00\x17\x00\x03\xc0\x10']
Bad pipe message: %s [b"%\xc8\xc9\x82J>\xa7\xd7\xde\xd0\xbeg\x0b\xcd\xe9m\x02\x93\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8

## GCANの実装

***

In [10]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms

datasets = datasets.MNIST(
    root='../data',
    train=True,
    download=True,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
)
batch_size = 50
dataloader = torch.utils.data.DataLoader(
    datasets,
    batch_size=batch_size,
    shuffle=True
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [11]:
import torch.nn as nn
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        in_ch = 1 #入力チャネル数
        start_ch = 128 #先頭層の出力チャネル数
        #識別きのネットワークを定義する
        self.layers = nn.ModuleList([
            #第１層: (bs, 1, 28, 28) -> (bs, 128, 14, 14)
            nn.Sequential(
                #畳み込み層
                nn.Conv2d(in_ch, start_ch, 4, 2, 1),
                nn.LeakyReLU(0.2, inplace=True)
            ),
            #第２層: (bs, 128, 14, 14) -> (bs, 256, 7, 7)
            nn.Sequential(
                #畳み込み層
                nn.Conv2d(start_ch, start_ch*2, 4, 2, 1),
                nn.BatchNorm2d(start_ch*2),
                nn.LeakyReLU(0.2, inplace=True)
            ),
            #第３層: (bs, 256, 7, 7) -> (bs, 512, 4, 4)
            nn.Sequential(
                #畳み込み層
                nn.Conv2d(start_ch*2, start_ch*4, 4, 2, 1),
                nn.BatchNorm2d(start_ch*4),
                nn.LeakyReLU(0.2, inplace=True)
            ),
            #第４層: (bs, 512, 4, 4) -> (bs, 1, 1, 1)
            nn.Sequential(
                #畳み込み層
                nn.Conv2d(start_ch*4, 1, 4, 1, 0),
                #最終出力には活性化関数のシグモイド関数を使用
                nn.Sigmoid()
            )
        ])
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


In [12]:
#生成器のクラスを定義
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        input_dim = 100 #入力データの次元数
        out_ch = 128 #最終層のチャンネル数
        img_ch = 1 #生成画像のチャンネル数
        self.layers = nn.ModuleList([
            #第１層: (bs, 100, 1, 1) -> (bs, 512, 4, 4)
            nn.Sequential(
                #畳み込み層
                nn.ConvTranspose2d(input_dim, out_ch*4, 4, 1, 0),
                nn.BatchNorm2d(out_ch*4),
                nn.ReLU(inplace=True)
            ),
            #第２層: (bs, 512, 4, 4) -> (bs, 256, 7, 7)
            nn.Sequential(
                #畳み込み層
                nn.ConvTranspose2d(out_ch*4, out_ch*2, 4, 2, 1),
                nn.BatchNorm2d(out_ch*2),
                nn.ReLU(inplace=True)
            ),
            #第３層: (bs, 256, 7, 7) -> (bs, 128, 14, 14)
            nn.Sequential(
                #畳み込み層
                nn.ConvTranspose2d(out_ch*2, out_ch, 4, 2, 1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True)
            ),
            #第４層: (bs, 128, 14, 14) -> (bs, 1, 28, 28)
            nn.Sequential(
                #畳み込み層
                nn.ConvTranspose2d(out_ch, img_ch, 4, 2, 1),
                #最終出力には活性化関数のシグモイド関数を使用
                nn.Tanh()
            )
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

### ネットワークの重みを初期化する関数

***

In [13]:
def weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02) #平均０、標準偏差０．０２の正規分布で初期化
        m.bias.data.fill_(0) #バイアスは0で初期化
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02) #平均１、標準偏差０．０２の正規分布で初期化
        m.bias.data.fill_(0) #バイアスは0で初期化

### 生成器をインスタンス化して重みを初期化する

***

In [15]:
from torchsummary import summary
#生成器Generator
generator = Generator().to(device)
#重みを初期化する
generator.apply(weight_init)
#生成機の生成器のサマリーを表示
summary(generator, (100, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 512, 4, 4]         819,712
       BatchNorm2d-2            [-1, 512, 4, 4]           1,024
              ReLU-3            [-1, 512, 4, 4]               0
   ConvTranspose2d-4            [-1, 256, 8, 8]       2,097,408
       BatchNorm2d-5            [-1, 256, 8, 8]             512
              ReLU-6            [-1, 256, 8, 8]               0
   ConvTranspose2d-7          [-1, 128, 16, 16]         524,416
       BatchNorm2d-8          [-1, 128, 16, 16]             256
              ReLU-9          [-1, 128, 16, 16]               0
  ConvTranspose2d-10            [-1, 1, 32, 32]           2,049
             Tanh-11            [-1, 1, 32, 32]               0
Total params: 3,445,377
Trainable params: 3,445,377
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forw